There are multiple questions that are going in my head.

1. does win rate change over time? why?

2. Al nasser was the champion of the 2018/2019 season, Al hilal is the 2019/2020 season. What makes a champ?

3. does formation matter?

4. how important is the coach?

5. does the main scorrer participation in small clubs make as much difference as big clubs?

6. does playing in small unpopular venues make any difference?

7. how does previous match rating affect the next match result?

8. Which is more significant, last match rating, or the last match against the opposing team rating?

9. Does playing in unpopular venues have an effect on peformance?



In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
%matplotlib inline

In [236]:
season2018 = pd.read_csv("Season2018.csv")
season2019 = pd.read_csv("Season2019.csv")
season2020 = pd.read_csv("Season2020.csv")

df = pd.concat([season2018, season2019, season2020]).reset_index(drop = True)

In [237]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30T18:40:00+03:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30T19:05:00+03:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30T19:15:00+03:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31T18:40:00+03:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31T18:55:00+03:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


let's first isolate a portion of the matches (test set) so that it doesn't affect our decision making in the exploration phase

* I'll isolate 20% of the data.
* and i'll isolate it so that we have 20% of each season in our test set
* I'll not use indices i'll use fixture_ids

In [238]:
print(df.shape)
print(season2018.shape)
print(season2019.shape)
print(season2020.shape)

(720, 19)
(240, 19)
(240, 19)
(240, 19)


In [239]:
np.random.seed(42)
season2018_test = np.random.permutation(season2018["fixture_id"])[:48]
season2019_test = np.random.permutation(season2019["fixture_id"])[:48]
season2020_test = np.random.permutation(season2020["fixture_id"])[:48]

In [240]:
df.season.unique()

array(['2018/2019', '2019/2020', '2020/2021'], dtype=object)

In [241]:
test_dict = {"2018/2019": season2018_test, "2019/2020": season2019_test, '2020/2021' : season2020_test}

In [242]:
test_fixtures = np.append(np.append(season2018_test, season2019_test), season2020_test)

In [243]:
len(test_fixtures)

144

In [244]:
test_fixtures[:10]

array([151719, 151701, 151790, 151805, 151800, 151867, 151929, 151783,
       151704, 151838])

# Initial data exploration & assessing

###### Let's start with the most basic operations

In [245]:
df.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,151695,Al-Ettifaq,Al-Raed,1.0,1.0,Prince Mohamed Bin Fahd Stadium,NaN,2018-08-30T18:40:00+03:00,2018/2019,L. Ramos,B. Hasi,4-2-3-1,4-2-3-1,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...","[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",1,Match Finished,7.00,6.98
1,151696,Al-Hazm,Al Wehda Club,0.0,0.0,King Abdul Aziz Stadium,NaN,2018-08-30T19:05:00+03:00,2018/2019,D. Isăilă,Fábio Carille,4-2-3-1,4-4-2,"[('M. Asselah', None), ('Alemão', None), ('Kha...","[('Mohamed Awad', None), ('Osama Hawsawi', Non...",1,Match Finished,6.97,6.77
2,151697,Ohod,Al-Nassr,1.0,2.0,Prince Mohammed bin Abdul Aziz Stadium,NaN,2018-08-30T19:15:00+03:00,2018/2019,F. Arce,D. Carreño,4-2-3-1,4-2-3-1,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...","[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,Match Finished,6.64,6.83
3,151698,Al-Qadisiyah FC,Al-Fateh,0.0,0.0,Prince Saud bin Jalawi Stadium,"Mark Clattenburg, England",2018-08-31T18:40:00+03:00,2018/2019,A. Stanojević,F. Al Jabal,4-4-2,4-2-3-1,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...","[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",1,Match Finished,6.96,7.01
4,151699,Al-Hilal Saudi FC,Al-Fayha,1.0,0.0,King Fahd Stadium,NaN,2018-08-31T18:55:00+03:00,2018/2019,Jorge Jesus,G. Costas,4-1-4-1,4-3-3,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...","[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",1,Match Finished,7.07,6.55


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fixture_id      720 non-null    int64  
 1   home_team       720 non-null    object 
 2   away_team       720 non-null    object 
 3   home_goals      720 non-null    float64
 4   away_goals      720 non-null    float64
 5   venue           720 non-null    object 
 6   referee         657 non-null    object 
 7   date            720 non-null    object 
 8   season          720 non-null    object 
 9   home_coach      678 non-null    object 
 10  away_coach      680 non-null    object 
 11  home_formation  703 non-null    object 
 12  away_formation  703 non-null    object 
 13  home_players    720 non-null    object 
 14  away_players    720 non-null    object 
 15  round           720 non-null    int64  
 16  status          720 non-null    object 
 17  H_avg_ratings   703 non-null    flo

* date column must be converted to datetime

In [247]:
df.describe()

,fixture_id,home_goals,away_goals,round,H_avg_ratings,A_avg_ratings
count,720.000000,720.00000,720.000000,720.000000,703.000000,703.000000
mean,357523.500000,1.60000,1.375000,15.500000,6.867653,6.813371
std,212028.201345,1.29662,1.183304,8.661458,0.234104,0.242154
min,151695.000000,0.00000,0.000000,1.000000,6.100000,5.850000
25%,151874.750000,1.00000,0.000000,8.000000,6.710000,6.660000
50%,273876.500000,1.00000,1.000000,15.500000,6.870000,6.820000
75%,635926.250000,2.00000,2.000000,23.000000,7.020000,6.970000
max,657882.000000,7.00000,6.000000,30.000000,7.570000,7.570000


we notice that the mean of the home team ratings is slightly greater than the away team which indicates that being home or not is a good predictor

In [248]:
df.date

0      2018-08-30T18:40:00+03:00
1      2018-08-30T19:05:00+03:00
2      2018-08-30T19:15:00+03:00
3      2018-08-31T18:40:00+03:00
4      2018-08-31T18:55:00+03:00
                 ...            
715    2021-05-30T21:00:00+03:00
716    2021-05-30T21:00:00+03:00
717    2021-05-30T21:00:00+03:00
718    2021-05-30T21:00:00+03:00
719    2021-05-30T21:00:00+03:00
Name: date, Length: 720, dtype: object

In [249]:
#Let's first do the perfrom basic operations 
df.isnull().sum()

fixture_id         0
home_team          0
away_team          0
home_goals         0
away_goals         0
venue              0
referee           63
date               0
season             0
home_coach        42
away_coach        40
home_formation    17
away_formation    17
home_players       0
away_players       0
round              0
status             0
H_avg_ratings     17
A_avg_ratings     17
dtype: int64

most missing values come from the referee column

* We have 63 missing referees, 42 home team coach, 40 away team coach, and 17 formations, home & away player ratings.

We can manually get the referee, coaches, formations. But sadly the ratings can only be collected through the api, and the api doesn't have this info yet, so we might look for another way to deal with this column.

In [250]:
null_data = df[df.isnull().any(axis=1)]

In [251]:
null_data.loc[null_data["H_avg_ratings"].isnull() == True]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
426,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,NaN,2020-08-09T19:30:00+03:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
439,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,P. Kralovec,2020-08-15T20:50:00+03:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
444,312934,Al-Adalah,Al Wehda Club,2.0,4.0,Prince Abdullah bin Jalawi Sports City Stadium,M. Conger,2020-08-20T18:50:00+03:00,2019/2020,G. Solinas,Essa Al Mehyani,NaN,NaN,"[('Ali Al Mazidi', None), ('Mortada Al Barrih'...","[('Abdullah Al Jadani', None), ('Botía', None)...",26,Match Finished,NaN,NaN
452,312945,Al-Hilal Saudi FC,Al-Faisaly FC,2.0,0.0,King Saud University Stadium,NaN,2020-08-25T19:00:00+03:00,2019/2020,R. Lucescu,Péricles Chamusca,NaN,NaN,"[('Abdullah Al Maiouf', None), ('Yasir Al Shah...","[('Mustafa Malayekah', None), ('Igor Rossi', N...",27,Match Finished,NaN,NaN
453,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,NaN,2020-08-25T19:05:00+03:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
454,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,NaN,2020-08-25T19:20:00+03:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
455,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,A. Treimanis,2020-08-25T20:50:00+03:00,2019/2020,Rui Vitória,G. Solinas,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
461,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,João Pedro Pinheiro,2020-08-30T18:55:00+03:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,NaN,NaN,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
464,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,Khaled Al Teris,2020-09-04T20:35:00+03:00,2019/2020,A. Chebbi,V. Milojević,NaN,NaN,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
465,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,M. Conger,2020-09-04T20:35:00+03:00,2019/2020,Jorge Simão,G. Solinas,NaN,NaN,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN


* we notice that when the formation column is missing, the ratings are also missing. and this data shortage only happend in the 2019/2020 season. the 2020/2021 season ratings & formations are all present which is a good sign. Future matches might not encounter such absence of data.

# Data Cleaning 

* Well first try to fill in the missing data that we can using google's serach. It might be time consuming but complete data is always a plus.


* The referee column shouldn't logically be a predictor, as each referee should maintain neutrality. and not be in the favor of any team. If used to predict it would just add noise to the prediction.
It also has many missing values so I'll just drop the column.

#### Referee

In [252]:
df.drop("referee", axis = 1, inplace=True)

##### Coaches

Let's check how coaches names are spelled so that we have consistent values.

In [253]:
for team in df.home_team.unique():
    print(f"{team} coaches: ")
    print([coach for coach in df.loc[df["home_team"] == team].home_coach.unique()])
    print("--------------------------------------------------")

Al-Ettifaq coaches: 
['L. Ramos', 'Sergio Piernas', 'Hélder Cristóvão', 'Khaled Al Atawi', nan]
--------------------------------------------------
Al-Hazm coaches: 
['D. Isăilă', 'André Gaspar']
--------------------------------------------------
Ohod coaches: 
['F. Arce', 'Paulo Alves', 'A. Souayah']
--------------------------------------------------
Al-Qadisiyah FC coaches: 
['A. Stanojević', 'I. Petev', 'Bandar Basryh', 'Y. Al Mannai', nan]
--------------------------------------------------
Al-Hilal Saudi FC coaches: 
['Jorge Jesus', 'Z. Mamić', 'Péricles Chamusca', 'R. Lucescu', nan, 'Rogério Micale', 'José Morais']
--------------------------------------------------
Al Shabab coaches: 
['M. Şumudică', 'J. Almirón', 'Sultan Al-Qanwi', 'Luis García', 'Pedro Caixinha', nan, 'Carlos Hernandez']
--------------------------------------------------
Al-Faisaly FC coaches: 
['M. Rednic', 'F. Al-Waraqah', 'Péricles Chamusca', 'Al Humaidy Al Otaibi', nan]
---------------------------------------

In [254]:
df.loc[480, ["home_coach", "away_coach"]] = ['Pedro Caixinha', "A. Chebbi"]
df.loc[482, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'Y. Al Mannai']
df.loc[483, ["home_coach", "away_coach"]] = ["R. Lucescu", 'M. Skibbe']
df.loc[493, ["home_coach", "away_coach"]] = ["Péricles Chamusca", 'José Garrido']
df.loc[495, ["home_coach", "away_coach"]] = ["Y. Ferrera", 'Fábio Carille']
df.loc[499, ["home_coach", "away_coach"]] = ['Ivo Vieira', "Péricles Chamusca"]
df.loc[502, ["home_coach", "away_coach"]] = ['Fábio Carille', 'V. Milojević']
df.loc[504, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'B. Hasi']
df.loc[508, ["home_coach", "away_coach"]] = ['Fábio Carille', "Patrice Carteron"]
df.loc[509, ["home_coach", "away_coach"]] = ['M. Skibbe', 'V. Milojević']
df.loc[521, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Y. Al Mannai']
df.loc[532, ["home_coach", "away_coach"]] = ['Ivo Vieira', 'Fábio Carille']
df.loc[538, ["home_coach", "away_coach"]] = ['Pedro Caixinha', 'Fábio Carille']
df.loc[542, ["home_coach", "away_coach"]] = ['Rui Vitória', 'V. Milojević']
df.loc[562, ["home_coach", "away_coach"]] = ["R. Lucescu", 'Pedro Caixinha']
df.loc[580, ["home_coach", "away_coach"]] = ['Fábio Carille', 'A. Chebbi']
df.loc[583, ["home_coach", "away_coach"]] = ['V. Milojević', 'R. Lucescu']
df.loc[595, ["home_coach", "away_coach"]] = ["Khaled Al Atawi", 'V. Milojević']
df.loc[605, ["home_coach", "away_coach"]] = ['V. Milojević', 'José Garrido']
df.loc[609, ["home_coach", "away_coach"]] = ['A. Horvat', "Patrice Carteron"]
df.loc[617, ["home_coach", "away_coach"]] = ['V. Milojević', 'Fábio Carille']
df.loc[622, ["home_coach", "away_coach"]] = ['A. Horvat', 'Carlos Hernandez']
df.loc[627, ["home_coach", "away_coach"]] = ['V. Milojević', 'Pablo Machín']
df.loc[631, ["home_coach", "away_coach"]] = ['Rogério Micale', 'Khaled Al Atawi']
df.loc[634, ["home_coach", "away_coach"]] = ['Carlos Hernandez', 'V. Milojević']
df.loc[639, ["home_coach", "away_coach"]] = ['A. Horvat', 'Rogério Micale']
df.loc[642, ["home_coach", "away_coach"]] = ['Péricles Chamusca', 'V. Milojević']
df.loc[646, ["home_coach", "away_coach"]] = ['A. Horvat', 'A. Chebbi']
df.loc[653, ["home_coach", "away_coach"]] = ['Rogério Micale', 'B. Hasi']
df.loc[654, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Mahmoud Al-Hadid']
df.loc[659, ["home_coach", "away_coach"]] = ['Péricles Chamusca', 'B. Hasi']
df.loc[660, ["home_coach", "away_coach"]] = ['Fábio Carille', 'Carlos Hernandez']
df.loc[699, ["home_coach", "away_coach"]] = ['José Morais', "L. Reghecampf"]
df.loc[707, ["home_coach", "away_coach"]] = ['A. Veselinović', 'K. Režić']
df.loc[712, ["home_coach", "away_coach"]] = ['Y. Al Mannai', 'A. Chebbi']
df.loc[713, ["home_coach", "away_coach"]] = ['K. Režić', 'N. El Maestro']
df.loc[714, ["home_coach", "away_coach"]] = ['F. Al-Ghamdi', 'F. Vercauteren']
df.loc[715, ["home_coach", "away_coach"]] = ['B. Hasi', "Y. Ferrera"]
df.loc[716, ["home_coach", "away_coach"]] = ['Carlos Hernandez', 'G. Donis']
df.loc[717, ["home_coach", "away_coach"]] = ['L. Reghecampf', "Khaled Al Atawi"]
df.loc[718, ["home_coach", "away_coach"]] = ['José Morais', 'Péricles Chamusca']
df.loc[719, ["home_coach", "away_coach"]] = ['Mano Menezes', 'Fábio Carille']


In [255]:
df.loc[((df["home_coach"].isnull() == True) | (df["away_coach"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings


* Now we have all our coaches present 

##### Formation column

In [256]:
df.loc[((df["home_formation"].isnull() == True) | (df["away_formation"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
426,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,2020-08-09T19:30:00+03:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
439,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,2020-08-15T20:50:00+03:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
444,312934,Al-Adalah,Al Wehda Club,2.0,4.0,Prince Abdullah bin Jalawi Sports City Stadium,2020-08-20T18:50:00+03:00,2019/2020,G. Solinas,Essa Al Mehyani,NaN,NaN,"[('Ali Al Mazidi', None), ('Mortada Al Barrih'...","[('Abdullah Al Jadani', None), ('Botía', None)...",26,Match Finished,NaN,NaN
452,312945,Al-Hilal Saudi FC,Al-Faisaly FC,2.0,0.0,King Saud University Stadium,2020-08-25T19:00:00+03:00,2019/2020,R. Lucescu,Péricles Chamusca,NaN,NaN,"[('Abdullah Al Maiouf', None), ('Yasir Al Shah...","[('Mustafa Malayekah', None), ('Igor Rossi', N...",27,Match Finished,NaN,NaN
453,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,2020-08-25T19:05:00+03:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
454,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,2020-08-25T19:20:00+03:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
455,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,2020-08-25T20:50:00+03:00,2019/2020,Rui Vitória,G. Solinas,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
461,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,2020-08-30T18:55:00+03:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,NaN,NaN,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
464,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,2020-09-04T20:35:00+03:00,2019/2020,A. Chebbi,V. Milojević,NaN,NaN,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
465,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,2020-09-04T20:35:00+03:00,2019/2020,Jorge Simão,G. Solinas,NaN,NaN,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN


In [257]:
df.home_formation.unique()

array(['4-2-3-1', '4-4-2', '4-1-4-1', '4-3-1-2', '4-3-3', '4-2-2-2',
       '4-3-2-1', '4-5-1', '3-4-3', '5-3-2', '3-4-2-1', '5-4-1',
       '4-1-3-2', '3-5-2', '4-4-1-1', '3-1-4-2', nan, '3-4-1-2'],
      dtype=object)

In [258]:
df.loc[426, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[439, ["home_formation", "away_formation"]] = ["4-4-2", "4-2-3-1"]
df.loc[444, ["home_formation", "away_formation"]] = ["4-3-3", "4-2-3-1"]
df.loc[452, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[453, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-3-3"]
df.loc[454, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[455, ["home_formation", "away_formation"]] = ["4-2-3-1", "3-1-4-2"]
df.loc[461, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[464, ["home_formation", "away_formation"]] = ["4-4-2", "4-3-3"]
df.loc[465, ["home_formation", "away_formation"]] = ["4-4-2", "4-2-3-1"]
df.loc[466, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[467, ["home_formation", "away_formation"]] = ["4-4-1-1", "4-2-3-1"]
df.loc[468, ["home_formation", "away_formation"]] = ["4-4-2", "3-4-3"]
df.loc[469, ["home_formation", "away_formation"]] = ["4-1-4-1", "4-2-3-1"]
df.loc[470, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]
df.loc[471, ["home_formation", "away_formation"]] = ["5-4-1", "4-4-2"]
df.loc[474, ["home_formation", "away_formation"]] = ["4-2-3-1", "4-2-3-1"]

In [259]:
df.loc[((df["home_formation"].isnull() == True) | (df["away_formation"].isnull() == True)) & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings


##### Ratings

Now the only missing data are present in the ratings columns which sadly can not be collected.
I'll fill it in with the average rating of the team in that specific season.


The average will be calculated based on the matches present in the training set. We shouldn't touch the test set.

In [260]:
df.loc[((df["H_avg_ratings"].isnull() == True) | (df["A_avg_ratings"].isnull() == True)) \
       & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
426,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,2020-08-09T19:30:00+03:00,2019/2020,D. Carreño,B. Hasi,4-2-3-1,4-2-3-1,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
439,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,2020-08-15T20:50:00+03:00,2019/2020,Rui Vitória,D. Carreño,4-4-2,4-2-3-1,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
444,312934,Al-Adalah,Al Wehda Club,2.0,4.0,Prince Abdullah bin Jalawi Sports City Stadium,2020-08-20T18:50:00+03:00,2019/2020,G. Solinas,Essa Al Mehyani,4-3-3,4-2-3-1,"[('Ali Al Mazidi', None), ('Mortada Al Barrih'...","[('Abdullah Al Jadani', None), ('Botía', None)...",26,Match Finished,NaN,NaN
452,312945,Al-Hilal Saudi FC,Al-Faisaly FC,2.0,0.0,King Saud University Stadium,2020-08-25T19:00:00+03:00,2019/2020,R. Lucescu,Péricles Chamusca,4-2-3-1,4-2-3-1,"[('Abdullah Al Maiouf', None), ('Yasir Al Shah...","[('Mustafa Malayekah', None), ('Igor Rossi', N...",27,Match Finished,NaN,NaN
453,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,2020-08-25T19:05:00+03:00,2019/2020,N. Zekri,V. Milojević,4-2-3-1,4-3-3,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
454,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,2020-08-25T19:20:00+03:00,2019/2020,Essa Al Mehyani,Vítor Campelos,4-2-3-1,4-2-3-1,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
455,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,2020-08-25T20:50:00+03:00,2019/2020,Rui Vitória,G. Solinas,4-2-3-1,3-1-4-2,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
461,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,2020-08-30T18:55:00+03:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,4-2-3-1,4-2-3-1,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
464,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,2020-09-04T20:35:00+03:00,2019/2020,A. Chebbi,V. Milojević,4-4-2,4-3-3,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
465,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,2020-09-04T20:35:00+03:00,2019/2020,Jorge Simão,G. Solinas,4-4-2,4-2-3-1,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN


In [261]:
null_index = df.loc[((df["H_avg_ratings"].isnull() == True) | (df["A_avg_ratings"].isnull() == True)) \
       & (df["status"] == "Match Finished")].index

In [262]:
df.groupby("home_team")["H_avg_ratings"].mean()

home_team
Abha                 6.850690
Al Baten             6.800333
Al Shabab            6.944318
Al Taawon            6.899333
Al Wehda Club        6.869048
Al-Adalah            6.746429
Al-Ahli Jeddah       6.947111
Al-Ain               6.788667
Al-Ettifaq           6.880909
Al-Faisaly FC        6.878409
Al-Fateh             6.786591
Al-Fayha             6.811034
Al-Hazm              6.805517
Al-Hilal Saudi FC    7.009318
Al-Ittihad FC        6.883182
Al-Nassr             6.968372
Al-Qadisiyah FC      6.794667
Al-Raed              6.794318
Dhamk                6.818966
Ohod                 6.767333
Name: H_avg_ratings, dtype: float64

In [263]:
for i in null_index:
    season = df.loc[i, "season"]
    season_df = df.loc[df["season"] == season]
#     index = [int(x) for x in np.where(season_df["fixture_id"].isin(test_dict[season])==True)[0]]
    season_df = season_df.loc[~season_df["fixture_id"].isin(test_dict[season])]
    team1 = df.loc[i, "home_team"]
    team2 = df.loc[i, "away_team"]
    home_rating = season_df.groupby("home_team")["H_avg_ratings"].mean()
    away_rating = season_df.groupby("away_team")["A_avg_ratings"].mean()

    df.loc[i, "H_avg_ratings"] = round((home_rating[team1] + away_rating[team1]) / 2, 2)
    df.loc[i, "A_avg_ratings"] = round((home_rating[team2] + away_rating[team2]) / 2, 2)



In [264]:
df.loc[((df["H_avg_ratings"].isnull() == True) | (df["A_avg_ratings"].isnull() == True)) \
       & (df["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings


###### Now we have dealt with all missing data

##### Fixing the date column

In [265]:
df.loc[0, "date"]

'2018-08-30T18:40:00+03:00'

In [266]:
df["date"] = [date[:-6].replace("T", " ") for date in df["date"]]

In [267]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S")


In [268]:
df.date.dtype

dtype('<M8[ns]')

## Dataframe structure

with the current structure, If we want to analyze the data the key feature to analyze around is the team (who is playing?) but the team is spread into 2 columns, home & away, I want to merge them in 1 column (team).

This would result in a new column (IsHome?) and it would double the number of rows of our dataframe, since each match would now have 2 rows, one where the home team is represented by the column (team) and one where the opponent is represented in the column team.


###### changing the strucutre of the data so that we can perform better visualizations and predictions

In [269]:
# since every entry now is based on home I'll create a feature called isHome and set it to 1
df["isHome"] = 1

In [270]:
len([x for x in range(0, len(df)*2, 2)])

720

In [271]:
len(df)

720

In [272]:
# space out the indices so that we can fill in the missing places with the new strucutre
df.index = [x for x in range(0, len(df) * 2, 2)]

In [273]:
df.index

Int64Index([   0,    2,    4,    6,    8,   10,   12,   14,   16,   18,
            ...
            1420, 1422, 1424, 1426, 1428, 1430, 1432, 1434, 1436, 1438],
           dtype='int64', length=720)

I'll change the order of the columns so that it is close to the desired format

In [274]:
df = df[["fixture_id", "home_team", "home_formation", "home_coach", "home_players", "isHome", "H_avg_ratings",
         "away_team", "away_formation", "away_coach", "away_players", "A_avg_ratings",
         "season", "date", "venue", "status", "round","home_goals", "away_goals"]]

In [275]:
for i in [x for x in range(1, len(df) * 2 + 1, 2)]:
    oe = df.loc[i-1] #oe stands for old entry
    df.loc[i] = [oe.fixture_id, oe.away_team, oe.away_formation,
                 oe.away_coach, oe.away_players, 0,oe.A_avg_ratings,
                 oe.home_team, oe.home_formation, oe.home_coach, oe.home_players,
                 oe.H_avg_ratings, oe.season, oe.date, oe.venue,
                 oe.status, oe["round"], oe.away_goals, oe.home_goals]
    

In [276]:
df.sort_index(inplace=True)

In [277]:
df.head(6)

,fixture_id,home_team,home_formation,home_coach,home_players,isHome,H_avg_ratings,away_team,away_formation,away_coach,away_players,A_avg_ratings,season,date,venue,status,round,home_goals,away_goals
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",6.98,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",7.00,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",6.77,2018/2019,2018-08-30 19:05:00,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",6.97,2018/2019,2018-08-30 19:05:00,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0
4,151697,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",6.83,2018/2019,2018-08-30 19:15:00,Prince Mohammed bin Abdul Aziz Stadium,Match Finished,1,1.0,2.0
5,151697,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,6.83,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",6.64,2018/2019,2018-08-30 19:15:00,Prince Mohammed bin Abdul Aziz Stadium,Match Finished,1,2.0,1.0


In [278]:
df.columns

Index(['fixture_id', 'home_team', 'home_formation', 'home_coach',
       'home_players', 'isHome', 'H_avg_ratings', 'away_team',
       'away_formation', 'away_coach', 'away_players', 'A_avg_ratings',
       'season', 'date', 'venue', 'status', 'round', 'home_goals',
       'away_goals'],
      dtype='object')

* Now we will change the columns names and order

In [279]:
df.head(1)

,fixture_id,home_team,home_formation,home_coach,home_players,isHome,H_avg_ratings,away_team,away_formation,away_coach,away_players,A_avg_ratings,season,date,venue,status,round,home_goals,away_goals
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.0,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",6.98,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0


In [280]:
# Now we will change the columns so that it aligns with our format

df.columns = ["fixture_id", "team", "formation", "coach", "players", "isHome", "rating",
              "opponent", "opponent_formation", "opponent_coach","opponent_players", "opponent_rating",
              "season", "date", "venue", "status", "round", "team_goals", "opponent_goals"]

In [281]:
df.head(2)

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,opponent_players,opponent_rating,season,date,venue,status,round,team_goals,opponent_goals
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",6.98,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",7.00,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0


* I'll initially go with this structure, we might get back to the old one later

In [282]:
df.team_goals.dtype

dtype('float64')

# Feature Engineering

1. Current Points of both teams (before match is played) 
2. Final game result (Win, Draw, Defeat)

In [283]:
df["pts"] = 0
df["opponent_pts"] = 0
df["result"] = ""

In [284]:
#current points & result

for season in df.season.unique():
    
    season_df = df.loc[(df["season"] == season)].copy()

    points = {}

    for team in season_df.team.unique():
        points[team] = 0
    
    
    for i in season_df.index:
        if i % 2 == 0 :
            Team1 = season_df.loc[i, "team"]
            Team2 = season_df.loc[i, "opponent"]


            season_df.loc[i,"pts"] = points[Team1]
            season_df.loc[i, "opponent_pts"] = points[Team2]

            season_df.loc[i+1,"pts"] = points[Team2]
            season_df.loc[i+1, "opponent_pts"] = points[Team1]


            if season_df.loc[i, "team_goals"] > season_df.loc[i, "opponent_goals"]:
                points[Team1] += 3
                season_df.loc[i, "result"] = "Win"
                season_df.loc[i+1, "result"] = "Defeat"
            
            elif season_df.loc[i, "team_goals"] == season_df.loc[i, "opponent_goals"]:
                points[Team1] += 1
                points[Team2] += 1
                season_df.loc[i, "result"] = "Draw"
                season_df.loc[i+1, "result"] = "Draw"

            else:
                points[Team2] += 3
                season_df.loc[i, "result"] = "Defeat"
                season_df.loc[i+1, "result"] = "Win"

        df.loc[df["season"] == season] = season_df




In [285]:
df.head(6)

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,season,date,venue,status,round,team_goals,opponent_goals,pts,opponent_pts,result
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,2018/2019,2018-08-30 18:40:00,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,...,2018/2019,2018-08-30 19:05:00,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0,0,0,Draw
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,...,2018/2019,2018-08-30 19:05:00,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0,0,0,Draw
4,151697,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,...,2018/2019,2018-08-30 19:15:00,Prince Mohammed bin Abdul Aziz Stadium,Match Finished,1,1.0,2.0,0,0,Defeat
5,151697,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,6.83,Ohod,4-2-3-1,F. Arce,...,2018/2019,2018-08-30 19:15:00,Prince Mohammed bin Abdul Aziz Stadium,Match Finished,1,2.0,1.0,0,0,Win


<hr>

3. last match rating
    * if match is in round 1, I'll fill the rating with the season's average rating of that team

In [286]:
np.where(df["fixture_id"].isin(test_fixtures))[0]

array([  12,   13,   18,   19,   20,   21,   30,   31,   32,   33,   36,
         37,   38,   39,   48,   49,   50,   51,   60,   61,   76,   77,
         90,   91,  110,  111,  132,  133,  138,  139,  158,  159,  172,
        173,  186,  187,  192,  193,  208,  209,  218,  219,  224,  225,
        226,  227,  228,  229,  250,  251,  254,  255,  286,  287,  300,
        301,  304,  305,  306,  307,  312,  313,  330,  331,  336,  337,
        344,  345,  350,  351,  352,  353,  364,  365,  370,  371,  384,
        385,  402,  403,  412,  413,  442,  443,  454,  455,  456,  457,
        460,  461,  466,  467,  470,  471,  474,  475,  480,  481,  484,
        485,  498,  499,  502,  503,  516,  517,  536,  537,  540,  541,
        546,  547,  560,  561,  588,  589,  590,  591,  596,  597,  616,
        617,  620,  621,  626,  627,  632,  633,  634,  635,  636,  637,
        640,  641,  648,  649,  656,  657,  662,  663,  670,  671,  678,
        679,  682,  683,  694,  695,  728,  729,  7

In [287]:
season_df.index

Int64Index([ 960,  961,  962,  963,  964,  965,  966,  967,  968,  969,
            ...
            1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439],
           dtype='int64', length=480)

In [288]:
df["last_performance"] = 0
df["opponent_last_performance"] = 0

In [289]:
for season in df.season.unique():
    
    season_df = df.loc[(df["season"] == season)].copy()
    
    average_rating = season_df.loc[~season_df["fixture_id"].isin(test_fixtures)].groupby("team")["rating"].mean()    

    for i in season_df.index:

        if season_df.loc[i, "round"] == 1:
            season_df.loc[i, "last_performance"] = round(average_rating[season_df.loc[i, "team"]], 2)
            season_df.loc[i, "opponent_last_performance"] = round(average_rating[season_df.loc[i, "opponent"]], 2)
        
        else:
            season_df.loc[i, "last_performance"] = season_df.loc[(season_df["round"] == season_df.loc[i, "round"] - 1) & 
                                                                 (season_df["team"] == season_df.loc[i, "team"])].rating.values[0]
            season_df.loc[i, "opponent_last_performance"] = season_df.loc[(season_df["round"] == season_df.loc[i, "round"] - 1) & 
                                                                 (season_df["opponent"] == season_df.loc[i, "opponent"])].opponent_rating.values[0]
    
    
    df.loc[df["season"] == season] = season_df
    




In [290]:
df.head(2)

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,venue,status,round,team_goals,opponent_goals,pts,opponent_pts,result,last_performance,opponent_last_performance
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw,6.74,6.81
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw,6.81,6.74


<hr>

3. last game against the other team performance
   * since each team plays only twice a season with the opposing team, it wouldn't be right to take the average of a season for the first match, like the (last performance feature). We don't have that much data. I'm thinking we can take the last performance even if it happened in the pervious season.
   * if it is the first match between the two teams, I'll impute the performances with the average rating of all their matches together across all seasons. 
   * Some teams are not present in all seasons. they might only play 2 matches agaisnt other teams over the hall dataset.

In [291]:
df.head()

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,venue,status,round,team_goals,opponent_goals,pts,opponent_pts,result,last_performance,opponent_last_performance
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw,6.74,6.81
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,Prince Mohamed Bin Fahd Stadium,Match Finished,1,1.0,1.0,0,0,Draw,6.81,6.74
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,...,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0,0,0,Draw,6.82,6.86
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,...,King Abdul Aziz Stadium,Match Finished,1,0.0,0.0,0,0,Draw,6.86,6.82
4,151697,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,...,Prince Mohammed bin Abdul Aziz Stadium,Match Finished,1,1.0,2.0,0,0,Defeat,6.72,7.01


In [292]:
df["last_against_performance"] = 0
df["opponent_last_against_performance"] = 0

In [293]:
match_tuples = []
for team in df.team.unique():
    for opponent in df.opponent.unique():
        if team != opponent:
            if team is not np.nan and opponent is not np.nan:
                match_tuples.append((team, opponent))

In [294]:
match_tuples[:10]

[('Al-Ettifaq', 'Al-Raed'),
 ('Al-Ettifaq', 'Al Wehda Club'),
 ('Al-Ettifaq', 'Al-Hazm'),
 ('Al-Ettifaq', 'Al-Nassr'),
 ('Al-Ettifaq', 'Ohod'),
 ('Al-Ettifaq', 'Al-Fateh'),
 ('Al-Ettifaq', 'Al-Qadisiyah FC'),
 ('Al-Ettifaq', 'Al-Fayha'),
 ('Al-Ettifaq', 'Al-Hilal Saudi FC'),
 ('Al-Ettifaq', 'Al-Ittihad FC')]

In [295]:
len(match_tuples)

380

In [306]:
for team, opponent in match_tuples:

    against_matches = df.loc[(df["team"] == team) & (df["opponent"] == opponent)].copy()
    
    for iteration, index in enumerate(against_matches.index):
        if iteration == 0:
            against_matches.loc[index, "last_against_performance"] = round(against_matches.rating.mean(), 2)
            against_matches.loc[index, "opponent_last_against_performance"] = round(against_matches.opponent_rating.mean(), 2)
        else:
            against_matches.loc[index, "last_against_performance"] = against_matches.iloc[iteration-1].rating
            against_matches.loc[index, "opponent_last_against_performance"] = against_matches.iloc[iteration-1].opponent_rating
            
    df.loc[(df["team"] == team) & (df["opponent"] == opponent)] = against_matches.copy()

In [311]:
df.head()

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,round,team_goals,opponent_goals,pts,opponent_pts,result,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,1,1.0,1.0,0,0,Draw,6.74,6.81,6.90,6.72
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,1,1.0,1.0,0,0,Draw,6.81,6.74,6.72,6.90
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,...,1,0.0,0.0,0,0,Draw,6.82,6.86,6.81,7.02
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,...,1,0.0,0.0,0,0,Draw,6.86,6.82,7.02,6.81
4,151697,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,...,1,1.0,2.0,0,0,Defeat,6.72,7.01,6.52,7.07


In [312]:
df.describe()

,fixture_id,isHome,rating,opponent_rating,round,team_goals,opponent_goals,pts,opponent_pts,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance
count,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000
mean,357523.500000,0.500000,6.840917,6.840917,15.500000,1.487500,1.487500,20.058333,20.058333,6.843597,6.843597,6.841076,6.841076
std,211954.516477,0.500174,0.237318,0.237318,8.658448,1.245918,1.245918,14.101614,14.101614,0.233344,0.233344,0.219271,0.219271
min,151695.000000,0.000000,5.850000,5.850000,1.000000,0.000000,0.000000,0.000000,0.000000,5.850000,5.850000,5.850000,5.850000
25%,151874.750000,0.000000,6.690000,6.690000,8.000000,1.000000,1.000000,8.000000,8.000000,6.700000,6.700000,6.710000,6.710000
50%,273876.500000,0.500000,6.850000,6.850000,15.500000,1.000000,1.000000,19.000000,19.000000,6.850000,6.850000,6.840000,6.840000
75%,635926.250000,1.000000,7.000000,7.000000,23.000000,2.000000,2.000000,29.000000,29.000000,7.000000,7.000000,6.980000,6.980000
max,657882.000000,1.000000,7.570000,7.570000,30.000000,7.000000,7.000000,69.000000,69.000000,7.570000,7.570000,7.570000,7.570000


<hr>

4. Points difference 

In [313]:
df["pts_difference"] = df["pts"] - df["opponent_pts"]

In [314]:
df[["pts", "opponent_pts", "pts_difference"]]

,pts,opponent_pts,pts_difference
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
1435,44,39,5
1436,58,36,22
1437,36,58,-22
1438,46,53,-7


<hr>

##### 5. Points difference from the closet team (how close is the team to moving up a rank in the standards)

##### 6. Cumulative points sum after the match is finished.

* To get the difference we should be able to know how the points of each team at that instnace of time. It's a good thing we have a cumulative point feature that we can just look at, but this feature represens the current points of the 2 teams before the match is played. we need another feature that represents the points after finishing the match

In [341]:
df["pts_after"] = df["pts"]
df["opponent_pts_after"] = df["opponent_pts"]

In [342]:
df.loc[(df["result"] == "Draw"), "pts_after"] += 1
df.loc[(df["result"] == "Draw"), "opponent_pts_after"] += 1


In [343]:
df.loc[(df["result"] == "Win"), "pts_after"] += 3
df.loc[(df["result"] == "Defeat"), "opponent_pts_after"] += 3

In [344]:
df.tail()

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,result,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,pts_diff_closest,opponent_pts_diff_closest
1435,657880,Al-Ettifaq,4-4-2,Khaled Al Atawi,"[(""Raïs M'Bolhi"", 'G'), ('Saleh Al Qumayzi', '...",0,6.89,Al-Ahli Jeddah,4-1-4-1,L. Reghecampf,...,Win,6.87,6.66,6.91,6.99,5,47,39,0,0
1436,657881,Al-Hilal Saudi FC,4-2-3-1,José Morais,"[('Abdullah Al-Mayouf', 'G'), ('Mohammed Al-Bu...",1,6.74,Al-Faisaly FC,4-2-3-1,Péricles Chamusca,...,Win,6.99,6.44,6.91,6.96,22,61,36,0,0
1437,657881,Al-Faisaly FC,4-2-3-1,Péricles Chamusca,"[('Mustafa Malaika', 'G'), ('Ali Majrashi', 'D...",0,6.55,Al-Hilal Saudi FC,4-2-3-1,José Morais,...,Defeat,6.44,6.99,6.96,6.91,-22,36,61,0,0
1438,657882,Al-Nassr,4-2-3-1,Mano Menezes,"[('Brad Jones', 'G'), ('Sultan Al-Ghannam', 'D...",1,6.60,Al-Ittihad FC,4-2-3-1,Fábio Carille,...,Defeat,6.97,7.00,6.76,6.77,-7,46,56,0,0
1439,657882,Al-Ittihad FC,4-2-3-1,Fábio Carille,"[('Marcelo Grohe', 'G'), ('Saud Abdulhamid', '...",0,6.78,Al-Nassr,4-2-3-1,Mano Menezes,...,Win,7.00,6.97,6.77,6.76,7,56,46,0,0


* Now let's get back to the difference feature. 
* to calculate the difference, we can do that by checking all other teams points before this match. we could use the date column to help us query them.

# Not working properly

In [345]:
df["pts_diff_closest"] = 0
df["opponent_pts_diff_closest"] = 0

In [384]:
df.loc[(df["season"] == "2018/2019") & (df["team"] == "Al-Nassr")]

,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,result,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,pts_diff_closest,opponent_pts_diff_closest
5,151697,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,6.83,Ohod,4-2-3-1,F. Arce,...,Win,7.01,6.72,7.07,6.52,0,3,0,1,1
22,151706,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,6.90,Al-Faisaly FC,4-4-2,M. Rednic,...,Win,6.83,6.99,6.83,6.72,2,6,1,3,1
33,151711,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,7.41,Al-Qadisiyah FC,4-2-2-2,A. Stanojević,...,Win,6.90,7.08,7.02,6.68,2,9,4,6,4
50,151720,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('C. Ramos', 'D'), ('Bruno...",1,6.85,Al Taawon,4-3-3,Pedro Emanuel,...,Win,7.41,7.01,6.90,6.64,4,12,5,9,5
74,151732,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,7.36,Al-Hazm,4-2-3-1,D. Isăilă,...,Win,6.85,6.97,7.13,6.66,7,15,5,12,5
83,151737,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,7.17,Al Baten,4-3-3,F. Vercauteren,...,Win,7.36,6.91,7.06,6.54,11,18,4,15,4
109,151750,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,6.85,Al-Fayha,3-4-3,S. Muslin,...,Draw,7.17,6.60,6.98,6.64,15,19,4,18,3
124,151758,Al-Nassr,4-3-3,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",1,6.59,Al-Ahli Jeddah,4-3-3,P. Guede,...,Defeat,6.85,7.31,6.86,6.94,3,19,19,19,16
141,151765,Al-Nassr,4-3-3,D. Carreño,"[('Waleed Abdullah', 'G'), ('C. Ramos', 'D'), ...",0,7.15,Al-Ettifaq,4-2-3-1,L. Ramos,...,Win,6.59,6.28,6.91,6.68,5,22,14,19,14
150,151771,Al-Nassr,4-1-4-1,Hélder Cristóvão,"[('Waleed Abdullah', 'G'), ('Bruno Uvini', 'D'...",1,7.12,Al Shabab,3-4-2-1,M. Şumudică,...,Win,7.15,6.82,6.94,6.80,6,25,16,22,16


In [351]:
df2 = df.copy()

In [390]:
df.pts_after.max()


72

In [396]:
match.pts

53

In [401]:
df["pts_after"].values

1

In [407]:
match.pts

53

In [409]:
match = df.loc[50]

In [410]:
match.pts

9

In [412]:
df.loc[df['date'] < match.date].copy()


,fixture_id,team,formation,coach,players,isHome,rating,opponent,opponent_formation,opponent_coach,...,result,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,pts_diff_closest,opponent_pts_diff_closest
0,151695,Al-Ettifaq,4-2-3-1,L. Ramos,"[(""R. M'Bolhi"", 'G'), ('R. Arias', 'D'), ('Hus...",1,7.00,Al-Raed,4-2-3-1,B. Hasi,...,Draw,6.74,6.81,6.90,6.72,0,1,1,0,0
1,151695,Al-Raed,4-2-3-1,B. Hasi,"[('A. Doukha', 'G'), ('H. Belkaroui', 'D'), ('...",0,6.98,Al-Ettifaq,4-2-3-1,L. Ramos,...,Draw,6.81,6.74,6.72,6.90,0,1,1,0,0
2,151696,Al-Hazm,4-2-3-1,D. Isăilă,"[('M. Asselah', None), ('Alemão', None), ('Kha...",1,6.97,Al Wehda Club,4-4-2,Fábio Carille,...,Draw,6.82,6.86,6.81,7.02,0,1,1,1,1
3,151696,Al Wehda Club,4-4-2,Fábio Carille,"[('Mohamed Awad', None), ('Osama Hawsawi', Non...",0,6.77,Al-Hazm,4-2-3-1,D. Isăilă,...,Draw,6.86,6.82,7.02,6.81,0,1,1,1,1
4,151697,Ohod,4-2-3-1,F. Arce,"[('Z. Laaroubi', 'G'), ('Hussein Abdul Ghani',...",1,6.64,Al-Nassr,4-2-3-1,D. Carreño,...,Defeat,6.72,7.01,6.52,7.07,0,0,3,1,1
5,151697,Al-Nassr,4-2-3-1,D. Carreño,"[('B. Jones', 'G'), ('Bruno Uvini', 'D'), ('Om...",0,6.83,Ohod,4-2-3-1,F. Arce,...,Win,7.01,6.72,7.07,6.52,0,3,0,1,1
6,151698,Al-Qadisiyah FC,4-4-2,A. Stanojević,"[('J. Duncan', 'G'), ('Adnan Falatah', None), ...",1,6.96,Al-Fateh,4-2-3-1,F. Al Jabal,...,Draw,6.77,6.79,6.72,6.96,0,1,1,0,0
7,151698,Al-Fateh,4-2-3-1,F. Al Jabal,"[('M. Koval', 'G'), ('M. Aguirregaray', 'D'), ...",0,7.01,Al-Qadisiyah FC,4-4-2,A. Stanojević,...,Draw,6.79,6.77,6.96,6.72,0,1,1,0,0
8,151699,Al-Hilal Saudi FC,4-1-4-1,Jorge Jesus,"[('Ali Al Habsi', 'G'), ('Botía', 'D'), ('Yasi...",1,7.07,Al-Fayha,4-3-3,G. Costas,...,Win,7.01,6.72,7.03,6.55,0,3,0,0,0
9,151699,Al-Fayha,4-3-3,G. Costas,"[('C. Bonilla', 'G'), ('Naldo', 'M'), ('Gêgê',...",0,6.55,Al-Hilal Saudi FC,4-1-4-1,Jorge Jesus,...,Defeat,6.72,7.01,6.55,7.03,0,0,3,0,0


In [424]:
min(abs(df.pts_after.values-match.pts))

0

In [421]:
df.pts_after.describe()

count    1440.000000
mean       21.445833
std        14.293600
min         0.000000
25%        10.000000
50%        20.000000
75%        31.000000
max        72.000000
Name: pts_after, dtype: float64

In [385]:
df["pts_after"]

0        1
1        1
2        1
3        1
4        0
        ..
1435    47
1436    61
1437    36
1438    46
1439    56
Name: pts_after, Length: 1440, dtype: int64

In [432]:
for i in range(len(df)):
    match = df.loc[i].copy()
    season_df = df.loc[df["season"] == match.season].copy()
    before_matches = season_df.loc[season_df['date'] < match.date].copy()
    try:
#         #returns the closest difference
        difference = min(before_matches["pts_after"].values, key=lambda x:abs(x-match.pts))
    except:
        #if this is the first match
        difference = 0

    df.loc[(df["fixture_id"] == match.fixture_id) & (df["team"] == match.team), "pts_diff_closest"] = difference
    df.loc[(df["fixture_id"] == match.fixture_id) & (df["team"] != match.team), "opponent_pts_diff_closest"] = difference

fixture_id                                                                      151931
team                                                                          Al-Nassr
formation                                                                      4-2-3-1
coach                                                                      Rui Vitória
players                              [('B. Jones', 'G'), ('Maicon', 'D'), ('Bruno U...
isHome                                                                               1
rating                                                                            6.92
opponent                                                                      Al Baten
opponent_formation                                                               5-3-2
opponent_coach                                                               M. Panait
opponent_players                     [('Adriano Facchini', 'G'), ('Lucas', 'D'), ('...
opponent_rating                            

In [357]:
df.describe()

,fixture_id,isHome,rating,opponent_rating,round,team_goals,opponent_goals,pts,opponent_pts,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,pts_diff_closest,opponent_pts_diff_closest
count,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000
mean,357523.500000,0.500000,6.840917,6.840917,15.500000,1.487500,1.487500,20.058333,20.058333,6.843597,6.843597,6.841076,6.841076,0.000000,21.445833,21.445833,10.038889,10.038889
std,211954.516477,0.500174,0.237318,0.237318,8.658448,1.245918,1.245918,14.101614,14.101614,0.233344,0.233344,0.219271,0.219271,10.557497,14.293600,14.293600,14.058199,14.058199
min,151695.000000,0.000000,5.850000,5.850000,1.000000,0.000000,0.000000,0.000000,0.000000,5.850000,5.850000,5.850000,5.850000,-42.000000,0.000000,0.000000,0.000000,0.000000
25%,151874.750000,0.000000,6.690000,6.690000,8.000000,1.000000,1.000000,8.000000,8.000000,6.700000,6.700000,6.710000,6.710000,-5.000000,10.000000,10.000000,0.000000,0.000000
50%,273876.500000,0.500000,6.850000,6.850000,15.500000,1.000000,1.000000,19.000000,19.000000,6.850000,6.850000,6.840000,6.840000,0.000000,20.000000,20.000000,0.000000,0.000000
75%,635926.250000,1.000000,7.000000,7.000000,23.000000,2.000000,2.000000,29.000000,29.000000,7.000000,7.000000,6.980000,6.980000,5.000000,31.000000,31.000000,19.000000,19.000000
max,657882.000000,1.000000,7.570000,7.570000,30.000000,7.000000,7.000000,69.000000,69.000000,7.570000,7.570000,7.570000,7.570000,42.000000,72.000000,72.000000,66.000000,66.000000


In [285]:
df.corr()

,fixture_id,isHome,rating,opponent_rating,round,goals,pts,opponent_pts,last_performance,opponent_last_performance,last_against_performance,opponent_last_against_performance,pts_difference,pts_after,opponent_pts_after,pts_diff_closest,opponent_pts_diff_closest
fixture_id,1.000000,0.000000e+00,0.014799,0.014799,0.068312,0.036591,5.499257e-02,5.499257e-02,0.013584,0.013584,-0.014845,-0.014845,0.000000,0.054209,0.054209,-0.058610,-0.058610
isHome,0.000000,1.000000e+00,0.115589,-0.115589,0.000000,0.092498,3.500974e-19,-7.001947e-19,-0.041221,0.041221,-0.119436,0.119436,0.000000,0.008754,-0.008754,0.015117,-0.015117
rating,0.014799,1.155893e-01,1.000000,-0.663554,-0.011581,0.447916,1.126085e-01,-1.097709e-01,0.124469,-0.083346,0.255523,-0.239012,0.294112,0.177828,-0.176545,-0.051498,0.086926
opponent_rating,0.014799,-1.155893e-01,-0.663554,1.000000,-0.011581,-0.747308,-1.097709e-01,1.126085e-01,-0.083346,0.124469,-0.239012,0.255523,-0.294112,-0.176545,0.177828,0.086926,-0.051498
round,0.068312,0.000000e+00,-0.011581,-0.011581,1.000000,0.066558,8.479046e-01,8.479046e-01,0.001576,0.001576,0.036171,0.036171,0.000000,0.837797,0.837797,0.125020,0.125020
goals,0.036591,9.249836e-02,0.447916,-0.747308,0.066558,1.000000,1.479159e-01,-1.124009e-02,0.060990,-0.088755,0.180839,-0.173523,0.210495,0.207450,-0.066683,-0.089839,0.034744
pts,0.054993,3.500974e-19,0.112608,-0.109771,0.847905,0.147916,1.000000e+00,7.200688e-01,0.186613,0.004021,0.190866,-0.113385,0.374120,0.995616,0.704155,-0.066377,0.112038
opponent_pts,0.054993,-7.001947e-19,-0.109771,0.112608,0.847905,-0.011240,7.200688e-01,1.000000e+00,0.004021,0.186613,-0.113385,0.190866,-0.374120,0.704155,0.995616,0.112038,-0.066377
last_performance,0.013584,-4.122062e-02,0.124469,-0.083346,0.001576,0.060990,1.866131e-01,4.021232e-03,1.000000,-0.011112,0.138642,-0.141002,0.242632,0.190604,-0.003386,-0.156087,0.020315
opponent_last_performance,0.013584,4.122062e-02,-0.083346,0.124469,0.001576,-0.088755,4.021232e-03,1.866131e-01,-0.011112,1.000000,-0.141002,0.138642,-0.242632,-0.003386,0.190604,0.020315,-0.156087
